#  Setup stuff: Connectivity

In [ ]:
import os

from katana import remote
from katana.remote import import_data


os.environ["KATANA_SERVER_ADDRESS"] = "localhost:8080"

my_client = remote.Client()

print(my_client)

In [ ]:
DB_NAME    = "my_db"
GRAPH_NAME = "my_graph"

print("--")

In [ ]:
#  DELETE ALL GRAPHS

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
      l_handle=my_client.get_database(name=l_database.name).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")

In [ ]:
#  DELETE ALL DATABASES

for l_database in my_client.databases():
   if (l_database.name != "default"):
      my_client.get_database(name=l_database.name).delete_database()
      print("--")

for l_database in my_client.databases():
   print("DB ID: ", l_database.database_id, "     DB Name: ", l_database.name)

In [ ]:
#  CREATE DATABASE

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)

In [ ]:
#  CREATE GRAPH
#
my_graph = my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=2)

print(my_graph)

In [ ]:
#  CONNECT TO GRAPH

for l_graph in my_client.get_database(name=DB_NAME).graphs_in_database():
   if (l_graph.name == GRAPH_NAME):
      my_graph=my_client.get_database(name=DB_NAME).get_graph_by_id(id=l_graph.graph_id)
         #
      break

print(my_graph)

# Setup stuff: Debugging

In [ ]:
#  LIST ALL GRAPHS

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "     DB GRAPH: ", l_graph.name, "    GRAPH VERSION: ", l_graph.version)

print("--")


In [ ]:
display(my_graph.num_nodes())
display(my_graph.num_edges())

In [ ]:
my_graph.query("CALL graph.schema() RETURN *")


In [ ]:
my_client.widgets().operations()


In [ ]:
# ##################################################################

In [ ]:

l_result = my_graph.query("""

   MATCH (n) - [ r ] -> (m)
   RETURN n, m, r
   
   """, contextualize=True)

l_result.view()


# Create the vertices/nodes, edges ..


In [ ]:
import pandas as pd

print("--")

In [ ]:

#  We use a larger data set than analytics.sssp()

df_airports = pd.DataFrame([
   ["MKE", "Milwaukee"               ,"Airport"],
   ["ORD", "Chicago O-Hare"          ,"Airport"],
   ["SJC", "San Jose"                ,"Airport"],
   ["DEN", "Denver"                  ,"Airport"],
      #
   ["MSP", "Minneapolis, St.Paul"    ,"Airport"],
   ["MSN", "Madison"                 ,"Airport"],
   ["GRB", "Green Bay"               ,"Airport"],
      #
   ["EGE", "Eagle, Vail"             ,"Airport"],
   ["COS", "Colorado Springs"        ,"Airport"],
   ["PUB", "Pueblo"                  ,"Airport"],
   ["ASE", "Aspen"                   ,"Airport"],
   ["FNL", "Fort Collins, Loveland"  ,"Airport"],
      #
   ], columns = ["airport_code", "airport_name", "LABEL"])

df_airports.head(20)


In [ ]:
#  Create nodes, airports


import contextlib

for l_each in df_airports.iterrows():
    
   l_airport_code = l_each[1][0]
   l_airport_name = l_each[1][1]
   l_label        = l_each[1][2]
    
   l_query = """

      CREATE ( n: {2} {{ airport_code: '{0}' }} ) 
      SET n.airport_name = '{1}' 
   
      """.format(l_airport_code, l_airport_name, l_label)
         
    
   l_result0 = my_graph.query(l_query)
        
   # print(".", end='')


print("  ")
print("--")


In [ ]:
#  What's in the vertices/nodes ..

l_result = my_graph.query("""
   MATCH ( n ) 
   RETURN n
   """)
print(l_result[0:30])


In [ ]:
#  Delete all edges, nodes for test reset

l_result = my_graph.query("""
   MATCH (n) - [r] -> (m)
   RETURN count(r)
   """)
display(print(l_result))

l_result = my_graph.query("""
   MATCH (n)
   RETURN count(n)
   """)
display(print(l_result))

l_result = my_graph.query("""
   MATCH (n) - [r] -> (m)
   DELETE r
   """)
display(print(l_result))

l_result = my_graph.query("""
   MATCH (n)
   DELETE n
   """)
display(print(l_result))

l_result = my_graph.query("""
   MATCH (n) - [r] -> (m)
   RETURN count(r)
   """)
display(print(l_result))

l_result = my_graph.query("""
   MATCH (n)
   RETURN count(n)
   """)
display(print(l_result))


In [ ]:
# ##################################################################

In [ ]:
#  Create the edge, flights ..

df_flights = pd.DataFrame([
   ["MKE", "ORD",   66, 1, "FLIES_TO"],
   ["ORD", "MKE",   66, 1, "FLIES_TO"],
      #
   ["ORD", "DEN",  886, 1, "FLIES_TO"],
   ["DEN", "ORD",  886, 1, "FLIES_TO"],
      #
   ["SJC", "DEN",  948, 1, "FLIES_TO"],                           #  Notice SJC flies to/from Denver
   ["DEN", "SJC",  948, 1, "FLIES_TO"],
      #
   ["SJC", "ORD", 1829, 1, "FLIES_TO"],                           #  Notice SJC flies to ORD, but not ORD to SJC
      #
      #
   ["MSN", "ORD",  135, 1, "FLIES_TO"],                           #  ORD <-->  MSN, MSP, GRB
   ["ORD", "MSN",  135, 1, "FLIES_TO"],
   ["MSP", "ORD",  334, 1, "FLIES_TO"],
   ["ORD", "MSP",  334, 1, "FLIES_TO"],
   ["GRB", "ORD",  196, 1, "FLIES_TO"],
   ["ORD", "GRB",  196, 1, "FLIES_TO"],
      #
   ["EGE", "DEN",  155, 1, "FLIES_TO"],                           #  DEN <-->  EGE, COS, PUB, ASE, FNL
   ["DEN", "EGE",  155, 1, "FLIES_TO"],
   ["COS", "DEN",   95, 1, "FLIES_TO"],
   ["DEN", "COS",   95, 1, "FLIES_TO"],
   ["PUB", "DEN",  133, 1, "FLIES_TO"],
   ["DEN", "PUB",  133, 1, "FLIES_TO"],
   ["ASE", "DEN",  125, 1, "FLIES_TO"],
   ["DEN", "ASE",  125, 1, "FLIES_TO"],
   ["FNL", "DEN",   58, 1, "BUSES_TO"],
   ["DEN", "FNL",   58, 1, "BUSES_TO"],
      #
   ], columns = ["START_ID", "END_ID", "DISTANCE", "NUM_HOPS", "TYPE"])

df_flights.head(30)


In [ ]:

for l_each in df_flights.iterrows():
    
   l_startid    = l_each[1][0]
   l_endid      = l_each[1][1]
   l_distance   = l_each[1][2]
   l_num_hops   = l_each[1][3]
   l_type       = l_each[1][4]
    
   l_query = """
   
      MATCH
         (n: Airport),
         (m: Airport)
      WHERE n.airport_code = '{0}'
      AND   m.airport_code = '{1}'
      CREATE (n) -[r: {4} {{ DISTANCE: {2}, NUM_HOPS: {3} }}]-> (m)

      """.format(l_startid, l_endid, l_distance, l_num_hops, l_type)
    
   l_result0 = my_graph.query(l_query)
        
   # print(".", end='')


print("  ")
print("--")


In [ ]:
#  What's in the edge .. 

l_result = my_graph.query("""

   MATCH ( n )  - [r] ->  (m)
   RETURN  n.airport_code AS FROM, m.airport_code AS TO, r.DISTANCE AS DISTANCE, r.NUM_HOPS AS NUM_HOPS
   
   """)
print(l_result[0:30])


#  Cool error,  (Future investigation)
#
#  OperationError: query 7gWTPQVzbyQAppfqipc3mfvn24uDxFmdWhhUwcYKsscr-5mvU4Dju6SVQ6kYxt error: 
#  (../../libquery/src/QueryClient.cpp:394): backtrace: (../../libquery/src/OpGraph.cpp:202):
#  failed to parse query: failed to parse query near    MATCH  [r: FLIES_TO] : Invalid input
#  '[': expected an identifier, shortestPath, allShortestPaths or '(' (CypherParser.cpp:100)

